### LLMs

***Large Language Models*** (LLMs) are ***advanced machine learning models*** that excel in a wide range of ***language-related tasks*** without needing task-specific fine tuning for every scenario, such as:
- text generation
- translation
- summarization
- question answering
- and more

### LLM Additional Capabilities
- ***Tool calling***: to interact with external services
- ***Structured output***:  to make a chat model respond in a structured format, such as JSON
- ***Multimodality***: to work with data other than text; for example, images, audio, and video.

### Chat Models
In LangChain, the chat model is a way to provide a consistent interface for working with LLMs from different providers while offering additional features for monitoring, debugging, and optimizing.

### Initialize Chat Models

- ***Install the integration package***

`pip install langchain-openai`

`pip install langchain-anthropic`

You can see a full list of integration packages here [API Reference](https://python.langchain.com/api_reference/langchain/chat_models/langchain.chat_models.base.init_chat_model.html)

In [1]:
from langchain.chat_models import init_chat_model
#pip install python-dotenv
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

### Basic Usage

In [3]:
model = init_chat_model(
    model = 'gpt-4o-mini',
    model_provider = 'openai',
    temperature = 0.3,
)

In [4]:
response = model.invoke('hello, my name is Mahdi')
print(response.content)

Hello, Mahdi! How can I assist you today?


In [5]:
print('Model Type: ', type(model))
print('Response Type: ', type(response))

Model Type:  <class 'langchain_openai.chat_models.base.ChatOpenAI'>
Response Type:  <class 'langchain_core.messages.ai.AIMessage'>


In [6]:
response.content

'Hello, Mahdi! How can I assist you today?'

### Calculating Completion Cost

In [7]:
response.usage_metadata

{'input_tokens': 14,
 'output_tokens': 12,
 'total_tokens': 26,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [8]:
openai_pricing = {
    'gpt-4o-mini': {
        'input': 0.15,
        'output': 0.6
    }
}

In [9]:
input_tokens = response.usage_metadata['input_tokens']
output_tokens = response.usage_metadata['output_tokens']

cost = (openai_pricing['gpt-4o-mini']['input']*input_tokens + openai_pricing['gpt-4o-mini']['output']*output_tokens) / 1000000
print(f"Total Cost: {cost:8f}$")

Total Cost: 0.000009$


### Configurable Model

In [21]:
configurable_model = init_chat_model(temperature=0.7)

In [24]:
response = configurable_model.invoke(
    "Hello, my name is Mahdi",
    config={
        "configurable":{
            "model":'gpt-4o-mini' #"claude-3-5-sonnet-latest"
        }
    }
)

In [25]:
response

AIMessage(content='Hello, Mahdi! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 14, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CQ88IyAUH36kvj9iYB0vFcWmobGYE', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f717b8fe-9bb4-4fa9-b64f-09d0dcc8b1c5-0', usage_metadata={'input_tokens': 14, 'output_tokens': 12, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [27]:
response.tool_calls

[]

### Tool Calling

- Tool calling allows a chat model to respond to a given prompt by "calling a tool".

- The model only generates the arguments to a tool, and actually running the tool (or not) is up to the user.

![tool_call](images/tool_call.png)

- Tool Calling is supported by many popular LLM providers. You can find a list here: [link](https://python.langchain.com/docs/integrations/chat/)

In [55]:
from pydantic import BaseModel, Field

In [91]:
class GetPopulation(BaseModel):
    """Get the current population in a given location"""
    location: str = Field(..., description="The city, e.g. San Francisco")

In [92]:
llm = init_chat_model(temperature=0.7)

In [93]:
llm_with_tools = llm.bind_tools([GetPopulation])

In [94]:
response = llm_with_tools.invoke(
    "What is the population of Madrid?",
    config={
        'configurable':{
            'model': 'gpt-4o'
        }
    }
)

In [95]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_F9yzqpLPgbnSR7daFsQRTgD6', 'function': {'arguments': '{"location":"Madrid"}', 'name': 'GetPopulation'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 63, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb3c3cb84d', 'id': 'chatcmpl-CQ8LNTuNSSYm09cyOVOVhUNUMbEEW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--75d12d58-87a8-49d1-8067-61874607fa5a-0', tool_calls=[{'name': 'GetPopulation', 'args': {'location': 'Madrid'}, 'id': 'call_F9yzqpLPgbnSR7daFsQRTgD6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 14, 'total_tokens': 77, 'input_token_details':

In [96]:
response.tool_calls

[{'name': 'GetPopulation',
  'args': {'location': 'Madrid'},
  'id': 'call_F9yzqpLPgbnSR7daFsQRTgD6',
  'type': 'tool_call'}]

In [97]:
response = llm_with_tools.invoke(
    "What is the population of Madrid in 2025?",
    config={
        'configurable':{
            'model': 'gpt-4o'
        }
    }
)

In [98]:
response

AIMessage(content="I currently don't have the capability to provide future population estimates, such as the population of Madrid in 2025. For future population projections, you might want to consult demographic studies or reports from relevant statistical agencies or organizations.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 67, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb3c3cb84d', 'id': 'chatcmpl-CQ8LVwIkoxzqAfLC9M8AOwvtb01JF', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f1add6d8-9131-4646-8d31-b1546456e4ac-0', usage_metadata={'input_tokens': 67, 'output_tokens': 45, 'total_tokens': 112, 'input_token_details': {'audio': 0, 'cache_read'

In [99]:
response.tool_calls

[]

In [ ]:
https://python.langchain.com/docs/how_to/tool_calling/